In [1]:
from Dataload import dataload
import torch.nn.functional as F

import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils

from src import detect_faces, show_bboxes
from PIL import Image
import numpy as np
from module import video_model
from constant import EMOTIPATH,EMOTIFACEPATH,EMOTIAUDIOPATH
import os
import pandas as pd
import time
from tqdm.notebook import tqdm
import cv2
from torchvision import transforms


In [2]:
Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Audio_Train=os.path.join(EMOTIAUDIOPATH,"Train_audio.csv")
Audio_Valid=os.path.join(EMOTIAUDIOPATH,"Val_audio.csv")



In [11]:
def num_correct(prediction,labels):
    correct=0
    for i,(pred_label,label) in enumerate(zip(prediction,labels)):
        if (pred_label.item()==label.item()):
            correct +=1
    return correct

class Audio_Data(Dataset):
    def __init__(self,csv_file,audio_path=None):
        self._table = pd.read_csv(csv_file,delimiter=' ')
        self._audio_table=pd.read_csv(audio_path)
    def __len__(self):
        return len(self._table)
    def __getitem__(self, idx):
        audio_feature=self._audio_table.loc[self._audio_table.file_name ==self._table.Vid_name[idx]+'.arff']
        audio_feature=pd.to_numeric(audio_feature.values[0][2:])
        labels = torch.from_numpy(np.array(self._table.Label[idx]))-1
        return (audio_feature,labels)
        

class AudioRecognition(nn.Module):
    
    def __init__(self,softmax=True,label=3):
        super(AudioRecognition, self).__init__()
        self.label=label
        self.fc1=nn.Linear(988,512)
        self.fc2=nn.Linear(512,128)
        self.fc3=nn.Linear(128,self.label)
        self.bn1 = nn.BatchNorm1d(num_features=512)
        self.bn2=nn.BatchNorm1d(num_features=128)
        self.relu = nn.ReLU()
        self.softmax=softmax
        
    def forward(self,x):
        x=self.relu(self.bn1(self.fc1(x)))
        x=self.relu(self.bn2((self.fc2(x))))
        x=(self.fc3(x))
        
        if self.softmax:
            return F.softmax(x,1)
        else:
            return self.relu(x)


In [12]:
def train(num_epochs,name,model,train_dataloader,valid_dataloader,optimizer,criterion):
    f = open(name+".txt",'a')
    start = time.time()
    #Triaining
    train_loss=[]
    valid_accuracy=[]
    model.train()
    for epochs in range(0,num_epochs):
        first=True
        model.train()
        correct=0
        total_samples=0
        avg_tloss=0
        print("Training Epoch: ", epochs+1,"\n")
        for i_batch, (audio_file,label) in tqdm(enumerate(train_dataloader)):
         
            batch_size=audio_file.size(0)
            optimizer.zero_grad()
            audio_file=audio_file.float().to(device)
            output=model(audio_file)
            loss=criterion(output,label.to(device))
            loss.backward()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size
            accuracy=correct/(total_samples)
            optimizer.step()
            true_label=label.detach().cpu()
            avg_tloss+=loss.item()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
            if (i_batch+1)%84==0:
               # print(label)
                print("Batch: ",i_batch+1,"/",len(train_dataloader))
                print("Batch Recognition loss: ", loss.item())

        print(conf_mat)
        avg_tloss=avg_tloss/len(train_dataloader)
        avg_taccuracy=correct/total_samples
        print("Average_Loss: ",avg_tloss)
        print("Average_Accuracy: ",avg_taccuracy)

        torch.save(model.state_dict(),name+".pth")
        print("Validation\n")
        model.eval()   
        correct=0
        total_samples=0
        avg_vloss=0
        first=True

        for i_batch, (audio_file,label) in (enumerate(valid_dataloader)):
         
            batch_size=audio_file.size(0)
            optimizer.zero_grad()
            audio_file=audio_file.float().to(device)
            
            output=model(audio_file)
            loss=criterion(output,label.to(device))
            avg_vloss+=loss.item()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size

            true_label=label.detach().cpu()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
        print(conf_mat)
        avg_vloss=avg_vloss/len(valid_dataloader)
        print(avg_vloss)
        avg_vaccuracy=correct/(total_samples)
        print("Accuracy: ", avg_vaccuracy)

        data = " %f,%f,%f,%f \n" % (avg_tloss,avg_taccuracy,avg_vloss,avg_vaccuracy)
        f.write(data)



    f.close()

In [13]:
model=AudioRecognition()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)
model.to(device)

AudioRecognition(
  (fc1): Linear(in_features=988, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=3, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)

In [14]:
model(torch.rand(3,988))

tensor([[0.4899, 0.1863, 0.3238],
        [0.3550, 0.2067, 0.4383],
        [0.2335, 0.3245, 0.4420]], grad_fn=<SoftmaxBackward>)

In [15]:
train_set=Audio_Data(Train_label,Audio_Train)
valid_set=Audio_Data(Val_labels,Audio_Valid)


In [16]:
train_dataloader = DataLoader(train_set, batch_size=32
                       , num_workers=0,shuffle=True)

valid_dataloader = DataLoader(valid_set, batch_size=32
                       , num_workers=0,shuffle=False)

In [19]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0005, betas=(0.5, 0.999))
num_epochs=50
criterion = nn.CrossEntropyLoss()  

model_name="pre_embedded_audio"

In [20]:
train(num_epochs,model_name,model,train_dataloader,valid_dataloader,optimizer,criterion)

Training Epoch:  1 



Batch:  84 / 84
Batch Recognition loss:  1.1238806247711182

[[643  94  69]
 [ 46 807  70]
 [ 51  88 793]]
Average_Loss:  0.7128347931873231
Average_Accuracy:  0.8429161969184517
Validation

[[ 92 123  84]
 [ 66 155  60]
 [ 34  63  89]]
1.1055021459857624
Accuracy:  0.4386422976501306
Training Epoch:  2 



Batch:  84 / 84
Batch Recognition loss:  0.5625830888748169

[[647  84  75]
 [ 52 817  54]
 [ 40  74 818]]
Average_Loss:  0.6934683521588644
Average_Accuracy:  0.8575723412251034
Validation

[[ 94 125  80]
 [ 78 150  53]
 [ 38  62  86]]
1.1054153442382812
Accuracy:  0.4308093994778068
Training Epoch:  3 



Batch:  84 / 84
Batch Recognition loss:  0.5566364526748657

[[651  82  73]
 [ 56 797  70]
 [ 39  76 817]]
Average_Loss:  0.6956836637996492
Average_Accuracy:  0.8511837655016911
Validation

[[ 99 106  94]
 [ 88 132  61]
 [ 37  51  98]]
1.1041720484693844
Accuracy:  0.42950391644908614
Training Epoch:  4 



Batch:  84 / 84
Batch Recognition loss:  1.134116530418396

[[652  84  70]
 [ 41 825  57]
 [ 36  66 830]]
Average_Loss:  0.6885307232538859
Average_Accuracy:  0.8669673055242391
Validation

[[ 95 111  93]
 [ 72 149  60]
 [ 35  54  97]]
1.0970219522714615
Accuracy:  0.4451697127937337
Training Epoch:  5 



...

Batch:  84 / 84
Batch Recognition loss:  0.9139281511306763

[[706  61  39]
 [ 30 858  35]
 [ 14  44 874]]
Average_Loss:  0.6390823381287711
Average_Accuracy:  0.9161969184517099
Validation

[[ 95 126  78]
 [ 65 167  49]
 [ 34  64  88]]
1.084600495795409
Accuracy:  0.45691906005221933
Training Epoch:  47 



Batch:  84 / 84
Batch Recognition loss:  0.687264621257782

[[699  62  45]
 [ 33 854  36]
 [ 15  43 874]]
Average_Loss:  0.6385915847051711
Average_Accuracy:  0.9120631341600902
Validation

[[112 106  81]
 [ 84 151  46]
 [ 42  56  88]]
1.0792651325464249
Accuracy:  0.45822454308093996
Training Epoch:  48 



Batch:  84 / 84
Batch Recognition loss:  0.7931199073791504

[[706  64  36]
 [ 31 866  26]
 [ 17  42 873]]
Average_Loss:  0.6349866709538868
Average_Accuracy:  0.9188275084554679
Validation

[[103 128  68]
 [ 74 161  46]
 [ 38  66  82]]
1.0848595102628071
Accuracy:  0.4516971279373368
Training Epoch:  49 



Batch:  84 / 84
Batch Recognition loss:  0.9103434681892395

[[707  57  42]
 [ 35 858  30]
 [ 16  38 878]]
Average_Loss:  0.6367891182502111
Average_Accuracy:  0.918075911311537
Validation

[[ 93 118  88]
 [ 63 158  60]
 [ 34  56  96]]
1.08849951873223
Accuracy:  0.45300261096605743
Training Epoch:  50 



Batch:  84 / 84
Batch Recognition loss:  1.2133604288101196

[[700  64  42]
 [ 34 852  37]
 [ 12  46 874]]
Average_Loss:  0.6449994288739704
Average_Accuracy:  0.9116873355881248
Validation

[[ 80 142  77]
 [ 50 184  47]
 [ 28  75  83]]
1.0875975464781125
Accuracy:  0.45300261096605743
